# **Azure AI Agents with Action Tools**

## Overview
This notebook demonstrates how to create and use AI agents with action tools in Azure AI Foundry. You'll learn how to enhance an AI agent's capabilities by integrating custom functions that can perform actions on behalf of the user, allowing the agent to interact with external systems and APIs.

## What are Action Tools?
Unlike knowledge tools that retrieve information (like Bing search in the previous notebook), action tools enable agents to perform specific functions and tasks:

- **Custom Python Functions**: Define actions the agent can execute
- **Tool Calling**: The agent determines when and how to use these functions
- **Action Execution**: The application handles the actual function calls
- **Response Integration**: Results from function calls are incorporated into agent responses

This creates a powerful pattern where AI agents can interface with virtually any system or API through custom code.

## 1. Setting Up the Environment

First, we'll import the necessary libraries and load environment variables from a `.env` file.
This provides access to our Azure AI Foundry project connection string and model configuration.

In [1]:
import os
import dotenv
dotenv.load_dotenv(".env")

True

### Creating the Azure AI Project Client

Now we'll establish a connection to our Azure AI Foundry project using the connection string from our environment variables. This client will be used to manage agents, threads, and function tools.

In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

## 2. Defining Action Tools

Action tools are custom Python functions that the agent can call to perform specific tasks. Here we'll define two functions:

1. **fetch_weather**: Retrieves weather information for a specific location
2. **check_network_status**: Provides telecommunications network status information

These functions would typically connect to real APIs or databases, but for demonstration purposes, we'll use mock data.

In [3]:
import json
from typing import Any, Callable, Set, Optional
    
def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # Mock API response.
    mock_weather_data = {
        "New York": "Sunny, 25°C",
        "London": "Cloudy, 18°C",
        "Tokyo": "Rainy, 22°C",
        "Stockholm": "Snowy, -5°C",
    }
    
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json

def check_network_status(location: str, service_type: Optional[str] = None) -> str:
    """
    Checks the status of telecommunications network in a specified location.
    
    :param location (str): The location to check network status for.
    :param service_type (str, optional): Type of service to check (e.g., "4G", "5G", "VoLTE").
                                         If not specified, returns status for all services.
    :return: Network status information as a JSON string.
    :rtype: str
    """
    # Mock network status data
    network_data = {
        "Stockholm": {
            "5G": "Operational - 98.7% coverage",
            "4G": "Operational - 99.9% coverage",
            "VoLTE": "Operational"
        },
        "Gothenburg": {
            "5G": "Partial outage - 85.3% coverage",
            "4G": "Operational - 99.5% coverage",
            "VoLTE": "Operational"
        },
        "New York": {
            "5G": "Operational - 92.1% coverage",
            "4G": "Operational - 99.7% coverage",
            "VoLTE": "Maintenance in progress"
        },
        "London": {
            "5G": "Operational - 90.5% coverage",
            "4G": "Operational - 99.8% coverage",
            "VoLTE": "Operational"
        }
    }
    
    # Check if the location exists in our mock data
    if location not in network_data:
        return json.dumps({"error": "Location not found in network database"})
    
    # Return data for the specific service type if provided
    if service_type:
        if service_type in network_data[location]:
            return json.dumps({
                "location": location,
                "service": service_type,
                "status": network_data[location][service_type],
                "timestamp": "2025-04-04T10:30:00Z"  # Using current date from context
            })
        else:
            return json.dumps({"error": f"Service type {service_type} not available in {location}"})
    
    # Return all service data for the location
    return json.dumps({
        "location": location,
        "services": network_data[location],
        "timestamp": "2025-04-04T10:30:00Z"  # Using current date from context
    })


# Statically defined user functions for fast reference
agent_functions: Set[Callable[..., Any]] = {
    fetch_weather,
    check_network_status
}


## 3. Defining Agent Instructions

System instructions guide how the agent behaves and uses its tools. Here we create instructions that specify:
- The agent's capabilities (weather information and network status)
- When and how to use each of the available functions
- How the agent should format and present information to users

These instructions serve as the agent's "operating manual" for processing queries.

In [4]:
system_message=(
    "You are an assistant with access to both weather information and telecommunications network status data. "
    "You can help with the following:\n"
    "1. Checking current weather conditions in various locations\n"
    "2. Providing network status information in different regions, including details about 4G, 5G, and VoLTE services\n\n"
    "For network status, you can check overall network conditions in a location or get specific information about "
    "a particular service like 5G coverage. Feel free to ask for clarification if the user's request is ambiguous."
)

## 4. Creating the Agent

Now we'll create the actual agent with our system instructions and function tools. The agent combines:
- A language model (specified in our environment variables)
- Our system instructions
- Access to the custom functions we defined earlier

This creates a capable assistant that can perform actions on behalf of the user.

In [5]:
from azure.ai.projects.models import FunctionTool

functions = FunctionTool(functions=agent_functions)

agent = project_client.agents.create_agent(
    name="my-action-tool-agent",
    model=os.getenv("chatModel"),
    instructions=system_message,
    tools=functions.definitions,
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_xSKyXK9sKSd2Ja0Lj9WMbEhE


## 5. Creating a Thread

Threads are conversation containers in the Agent API. Each thread:
- Holds the history of messages between user and agent
- Maintains conversation state and context
- Enables multi-turn conversations

Think of a thread as a dedicated, persistent chat session for a specific conversation.

In [6]:
thread = project_client.agents.create_thread()

## 6. Adding a Message to the Thread

Now we'll add our first user message to the thread. This message will be processed by the agent when we run it.
We're asking about 5G network status in Stockholm, which will require the agent to use the `check_network_status` function.

In [7]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="How's the 5G network in Stockholm?",
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_0YNOEtyOAfUd2QjK1aOHn6ux


## 7. Running the Agent with Tool Execution

This section demonstrates the core workflow for agents with action tools:

1. **Create a Run**: Start the agent processing on our thread
2. **Monitor Status**: Check if the agent needs to call functions
3. **Execute Tool Calls**: When needed, run the appropriate function
4. **Submit Tool Outputs**: Return the function results to the agent
5. **Continue Processing**: Let the agent incorporate results into its response

This orchestration pattern allows the agent to seamlessly interact with external systems and APIs.

In [8]:
import time
from azure.ai.projects.models import RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput

run = project_client.agents.create_run(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, ID: {run.id}")

while run.status in ["queued", "in_progress", "requires_action"]:
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
        # When the status is requires_action, your code is responsible for calling the function tools.
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        if not tool_calls: # If no tool calls are provided, cancel the run.
            print("No tool calls provided - cancelling run")
            project_client.agents.cancel_run(thread_id=thread.id, run_id=run.id)
            break

        tool_outputs = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredFunctionToolCall):
                try:
                    print(f"Executing tool call: {tool_call}")
                    output = functions.execute(tool_call)
                    # If the output is not a string, convert it to JSON string
                    if not isinstance(output, str):
                        output = json.dumps(output)
                    tool_outputs.append(
                        ToolOutput(
                            tool_call_id=tool_call.id,
                            output=output,
                        )
                    )
                except Exception as e:
                    print(f"Error executing tool_call {tool_call.id}: {e}")

        print(f"Tool outputs: {tool_outputs}")
        if tool_outputs:
            project_client.agents.submit_tool_outputs_to_run(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
            )
    print(f"Current run status: {run.status}")
print(f"Run completed with status: {run.status}")

Created run, ID: run_zgFLYGio4vxIZvOgDHAf87mn
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Executing tool call: {'id': 'call_gpYlo9aUiup6rL3jVCo5b6mO', 'type': 'function', 'function': {'name': 'check_network_status', 'arguments': '{"location":"Stockholm","service_type":"5G"}'}}
Tool outputs: [{'tool_call_id': 'call_gpYlo9aUiup6rL3jVCo5b6mO', 'output': '{"location": "Stockholm", "service": "5G", "status": "Operational - 98.7% coverage", "timestamp": "2025-04-04T10:30:00Z"}'}]
Current run status: requires_action
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: completed
Run completed with status: completed


### Viewing Thread Messages

Now let's examine the conversation thread to see the agent's response. The `pretty_print_thread_messages` helper function formats the conversation in a readable way, showing both user queries and agent responses that incorporate information from the function call.

In [9]:
from utils.helper import pretty_print_thread_messages

messages = project_client.agents.list_messages(thread_id=thread.id)
pretty_print_thread_messages(messages)


👤 USER
------------------------------------------------------------


How's the 5G network in Stockholm?


🤖 ASSISTANT
------------------------------------------------------------


The 5G network in Stockholm is operational with a coverage of 98.7%.

## 8. Continuing the Conversation

Let's add a follow-up question to demonstrate how the agent maintains context across multiple turns. This time we'll ask about the weather in Stockholm, which will require the agent to use the `fetch_weather` function.

In [10]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="what's the weather?",
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_EhMVEfOw6rN3DPu1j8QJ2djq



### Processing the Follow-up Question

Now we'll run the agent again to process our follow-up question about the weather. The agent will:
1. Understand the query is about weather rather than network status
2. Use the appropriate `fetch_weather` function
3. Incorporate the weather data into its response

This demonstrates how an agent can intelligently select the right tool for each user request.

In [11]:
import time
from azure.ai.projects.models import RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput

run = project_client.agents.create_run(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, ID: {run.id}")

while run.status in ["queued", "in_progress", "requires_action"]:
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
        # When the status is requires_action, your code is responsible for calling the function tools.
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        if not tool_calls: # If no tool calls are provided, cancel the run.
            print("No tool calls provided - cancelling run")
            project_client.agents.cancel_run(thread_id=thread.id, run_id=run.id)
            break

        tool_outputs = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredFunctionToolCall):
                try:
                    print(f"Executing tool call: {tool_call}")
                    output = functions.execute(tool_call)
                    # If the output is not a string, convert it to JSON string
                    if not isinstance(output, str):
                        output = json.dumps(output)
                    tool_outputs.append(
                        ToolOutput(
                            tool_call_id=tool_call.id,
                            output=output,
                        )
                    )
                except Exception as e:
                    print(f"Error executing tool_call {tool_call.id}: {e}")

        print(f"Tool outputs: {tool_outputs}")
        if tool_outputs:
            project_client.agents.submit_tool_outputs_to_run(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
            )
    print(f"Current run status: {run.status}")
print(f"Run completed with status: {run.status}")

Created run, ID: run_AEtiK6WghiXXYtYI2qFuLdAt

Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Executing tool call: {'id': 'call_tlDfVbffWsMauOqPV2kZ2apY', 'type': 'function', 'function': {'name': 'fetch_weather', 'arguments': '{"location":"Stockholm"}'}}
Tool outputs: [{'tool_call_id': 'call_tlDfVbffWsMauOqPV2kZ2apY', 'output': '{"weather": "Snowy, -5\\u00b0C"}'}]
Current run status: requires_action
Current run status: queued
Current run status: queued
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: completed
Run completed with status: completed


### Viewing the Complete Conversation

Let's see the full conversation thread, which now includes both our network status query and weather query, along with the agent's responses that incorporate data from the function calls.

In [12]:
from utils.helper import pretty_print_thread_messages

messages = project_client.agents.list_messages(thread_id=thread.id)
pretty_print_thread_messages(messages)


👤 USER
------------------------------------------------------------


How's the 5G network in Stockholm?


🤖 ASSISTANT
------------------------------------------------------------


The 5G network in Stockholm is operational with a coverage of 98.7%.


👤 USER
------------------------------------------------------------


what's the weather?


🤖 ASSISTANT
------------------------------------------------------------


The weather in Stockholm is currently snowy, with a temperature of -5°C.

## 9. Cleanup

When we're finished with our agent and thread, it's good practice to clean up these resources. This helps manage resource usage and keeps your environment tidy.

In a production environment, you might retain threads for longer periods to maintain conversation history, but for this demonstration we'll delete both the agent and thread.

In [13]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

# Delete Thread when done
project_client.agents.delete_thread(thread_id=thread.id)
print(f"thread: {thread.id} deleted")

Deleted agent

thread: thread_prL0KeyvYaZx0T3q3rIDdoTi deleted
